# Практическое задание №8 по теме "Дисперсионный анализ. Метод главных компонент. Логистическая регрессия".

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from warnings import filterwarnings

filterwarnings("ignore")

## Задание 1

Используя данные о хоккеистах, проверьте, является ли среди хоккеистов из Финляндии, Норвегии и Дании значимым отличие: 
* а) роста, 
* б) BMI.

___Решение___

Рассмотрим информацию о датасете хоккеистов.

In [2]:
hockey_df = pd.read_csv('hockey_players.csv', encoding='ISO-8859-1')
hockey_df.head()

,year,country,no,name,position,side,height,weight,birth,club,age,cohort,bmi
0,2001,RUS,10,tverdovsky oleg,D,L,185,84.0,1976-05-18,anaheim mighty ducks,24.952772,1976,24.543462
1,2001,RUS,2,vichnevsky vitali,D,L,188,86.0,1980-03-18,anaheim mighty ducks,21.119781,1980,24.332277
2,2001,RUS,26,petrochinin evgeni,D,L,182,95.0,1976-02-07,severstal cherepovetal,25.229295,1976,28.680111
3,2001,RUS,28,zhdan alexander,D,R,178,85.0,1971-08-28,ak bars kazan,29.675565,1971,26.827421
4,2001,RUS,32,orekhovsky oleg,D,R,175,88.0,1977-11-03,dynamo moscow,23.490760,1977,28.734694


Избавляемся от дубликатов и лишних признаков, оставляем только хоккеистов из Финляндии, Норвегии и Дании.

In [3]:
hockey_df.drop_duplicates(subset=['name'], inplace=True)
hockey_df = hockey_df[['year','country','name','height','bmi']]
hockey_df = hockey_df[(hockey_df['country'] == 'FIN') | (hockey_df['country'] == 'NOR') | (hockey_df['country'] == 'DEN')]
hockey_df.head()

,year,country,name,height,bmi
120,2001,FIN,kiprusoff marko,184,26.287807
121,2001,FIN,nummelin petteri,178,26.196187
122,2001,FIN,timonen kimmo,178,27.774271
123,2001,FIN,niemi antti-jussi,185,25.420015
124,2001,FIN,salo sami,192,26.855469


Указанные в задании гипотезы можно проверить с помощью однофакторного дисперсионного анализа.

__а) Проверяем является ли среди хоккеистов из Финляндии, Норвегии и Дании значимым отличие роста__

1. Нулевой гипотезой $H_{0}$ считаем гипотезу о равенстве среднего роста по каждой группе (хоккеисты определенной страны), а альтернативной гипотезой $H_{1}$ - отличие роста.

2. Для проверки гипотез в дисперсионном анализе используется __F-критерий Фишера__. Используемая статистика представляет из себя отношение дисперсии между уровнями к дисперсии внутри уровней.

Пусть в каждой выборке $Y_i$ содержится $n_i$ элементов. Обозначим через $Y$ объединение всех выборок, т.е. выборку размера $n = n_1 + \dots + n_k$.

Рассмотрим две суммы квадратов:
$$SS_b = \displaystyle\sum_{i=1}^k \left( \overline{Y_i} - \overline{Y} \right)^2 \cdot n_i, \:\:
SS_w = \displaystyle\sum_{i=1}^k \displaystyle\sum_{j=1}^{n_i} \left( y_{ij} - \overline{Y_i} \right)^2,$$
где $y_{ij}$ — $j$-й элемент $i$-й выборки.

Первая сумма — отклонения между группами («b» от слова Between — между), вторая — отклонения внутри групп («w» от слова Within — внутри).

По этим значениям вычисляются соответствующие несмещённые оценки дисперсий:
$$\sigma_b^2 = \dfrac{SS_b}{k - 1}, \:\: \sigma_w^2 = \dfrac{SS_w}{n - k}$$

Итак, статистика для проверки гипотезы $H_0$:
$$F = \dfrac{\sigma_b^2}{\sigma_w^2}$$

3. Уровень значимости $\alpha = 0.05$

4. Критическая область $\Omega_\alpha$

В предположении верности гипотезы $H_0$ статистика $F$ имеет распределение Фишера с параметрами $k_1 = k - 1$, $k_2 = n - k$. Критическая область здесь правосторонняя:
$$\Omega_\alpha = \left( t_{1 - \alpha, \: k_1, k_2}, \infty \right),$$
где $t_{x, \: k_1, k_2}$ — квантиль порядка $x$ для распределения Фишера с параметрами $k_1$, $k_2$.

5. Расчет статистики

In [4]:
height_fin = hockey_df.loc[(hockey_df['country'] == 'FIN'), 'height'].values
height_nor = hockey_df.loc[(hockey_df['country'] == 'NOR'), 'height'].values
height_den = hockey_df.loc[(hockey_df['country'] == 'DEN'), 'height'].values

In [5]:
statistics, pvalue = stats.f_oneway(height_fin, height_nor, height_den)
print(f"Значение статистики: {np.round(statistics, 8)}\nP-значение: {np.round(pvalue, 8)}")

Значение статистики: 3.5504111
P-значение: 0.02956354


__P-значение меньше уровня значимости__ $\alpha=0.05$, следовательно, нулевая $H_{0}$ гипотеза отвергается, а отличие роста между выборками хоккеистов из Финляндии, Норвегии и Дании признается значимым.

__б) Проверяем является ли среди хоккеистов из Финляндии, Норвегии и Дании значимым отличие BMI__

1. Нулевой гипотезой $H_{0}$ считаем гипотезу о равенстве среднего индекса массы тела BMI по каждой группе (хоккеисты определенной страны), а альтернативной гипотезой $H_{1}$ - отличие индекса массы тела BMI.

2. Cтатистика для проверки гипотезы $H_0$:
$$F = \dfrac{\sigma_b^2}{\sigma_w^2}$$

3. Уровень значимости $\alpha = 0.05$

4. Критическая область $\Omega_\alpha$

В предположении верности гипотезы $H_0$ статистика $F$ имеет распределение Фишера с параметрами $k_1 = k - 1$, $k_2 = n - k$. Критическая область здесь правосторонняя:
$$\Omega_\alpha = \left( t_{1 - \alpha, \: k_1, k_2}, \infty \right),$$
где $t_{x, \: k_1, k_2}$ — квантиль порядка $x$ для распределения Фишера с параметрами $k_1$, $k_2$.

5. Расчет статистики

In [6]:
bmi_fin = hockey_df.loc[(hockey_df['country'] == 'FIN'), 'bmi'].values
bmi_nor = hockey_df.loc[(hockey_df['country'] == 'NOR'), 'bmi'].values
bmi_den = hockey_df.loc[(hockey_df['country'] == 'DEN'), 'bmi'].values

In [7]:
statistics, pvalue = stats.f_oneway(bmi_fin, bmi_nor, bmi_den)
print(f"Значение статистики: {np.round(statistics, 8)}\nP-значение: {np.round(pvalue, 8)}")

Значение статистики: 8.47342551
P-значение: 0.00024614


__P-значение меньше уровня значимости__ $\alpha=0.05$, следовательно, нулевая $H_{0}$ гипотеза отвергается, а отличие индекса массы тела BMI между выборками хоккеистов из Финляндии, Норвегии и Дании признается значимым.

___Ответ:___ 

__а)__ Отличие роста среди хоккеистов из Финляндии, Норвегии и Дании является значимым<br>
__б)__ Отличие BMI среди хоккеистов из Финляндии, Норвегии и Дании является значимым

---